Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

# Lab3 - Modularity, Parameters, and Decoupling
> Labs will be due each week before the homeworks. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
val path = System.getProperty("user.dir") + "/resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chisel3.tester.RawTester.test

## Problem 1 (2 pts) - Chisel without Modules - Companion Objects
> Throughout our labs so far, we have typically wrapped our code in a class that extends `Module` anytime we wished to write Chisel code. We can actually use standard Scala classes/objects/defs to return Chisel components as long as they are eventually used in a `Module`. The best example of this is using _companion object_ to provide a -factory method_. These are very useful for tidying up io connections.
>
> Write an `apply` method that creates an instance of an `ManyConnections` module, connects all the io, and then returns the module's output as a `Bool`.


In [ ]:
class ManyConnections extends Module {
    val io = IO(new Bundle {
        val in0  = Input(Bool())
        val in1  = Input(Bool())
        val in2  = Input(Bool())
        val in3  = Input(Bool())
        val in4  = Input(Bool())
        val out = Output(Bool())
    })
    io.out := io.in0 & io.in1 & io.in2 & io.in3 & io.in4 
}

object ManyConnections {
// YOUR CODE HERE
???
}

In [ ]:
class UseManyConn extends Module {
    val io = IO(new Bundle {
        val in0 = Input(Bool())
        val in1 = Input(Bool())
        val in2 = Input(Bool())
        val in3 = Input(Bool())
        val in4 = Input(Bool())   
        val out = Output(Bool())
    })
    
    val one = ManyConnections(true.B, true.B, true.B, true.B, true.B)
    val and = ManyConnections(io.in0, io.in1, io.in2, io.in3, io.in4)
    
//     Hopefully above is more appealing than multiples of these:
    
//     val m0 = Module(new ManyConnections)
//     m0.io.in0 := io.in0
//     m0.io.in1 := io.in1
//     m0.io.in2 := io.in2
//     m0.io.in3 := io.in3
//     m0.io.in4 := io.in4
//     val and = m0.io.out
    io.out := one & and
    
}

def testManyConnections: Boolean = {
    test(new ManyConnections) { dut =>
        dut.io.in0.poke(true.B)
        dut.io.in1.poke(true.B)
        dut.io.in2.poke(true.B)
        dut.io.in3.poke(true.B)
        dut.io.in4.poke(true.B)
        dut.io.out.expect(true.B)
        
        
        dut.io.in0.poke(false.B)
        dut.io.in1.poke(true.B)
        dut.io.in2.poke(true.B)
        dut.io.in3.poke(true.B)
        dut.io.in4.poke(true.B)
        dut.io.out.expect(false.B)
        
    }
    true
}
assert(testManyConnections)

## Problem 2 (5 pts) - Case Classes
> Case Classes are very useful for packaging our parameters to the same location. Implement a case class `ROMParams` to complete the design below. When completed, the `ROM` module can be created with a `Seq` (passed through the case class), and the generator will appropriately size the various bitwidths and instantiate a read-only memory. Based on the code provided, you should be able to infer the missing field names.

In [ ]:
case class ROMParams(data: Seq[Int]) {
    val numElems = data.size
    val largestElem = data.max
    var dataInChiselT = Seq[UInt]()
    for (x <- data) {
        dataInChiselT = dataInChiselT :+ x.U
    }
    // YOUR CODE HERE
    ???
}

class ROMIO (p: ROMParams) extends Bundle {
    val sel = Input(UInt(p.addrWidth.W))
    val out = Output(UInt(p.elemWidth.W))
    override def cloneType = (new ROMIO(p)).asInstanceOf[this.type]

}

class ROM (p: ROMParams) extends Module {
    val io = IO(new ROMIO(p))
    // YOUR CODE HERE
    ???
}

In [ ]:
def testROM(l: Seq[Int]): Boolean = {
    val p = ROMParams(l)
    test(new ROM(p)) { dut =>
        for (i <- 0 until l.size) {
            dut.io.sel.poke(i.U)
            dut.io.out.expect(l(i).U)
        }
        assert(p.addrWidth == log2Ceil(l.size + 1))
        assert(p.elemWidth == log2Ceil(l.max + 1))
    }
    true
}
assert(testROM((0 until 5).toSeq))
assert(testROM((20 until 31).toSeq))

## Problem 3 (3 pts) - Chisel without Modules - Accumulator
> We saw in class how to create a counter without the use of `Module`. Make a class `Accumulator` without the use of `Module`. This class should have a register of type `UInt` to store the accumulated `data` values. When the `rst` signal is high, the accumulated value should reset to `0.U`. 

In [ ]:
object Accumulator {
    def apply(width: Int, data: UInt, rst: Bool) = {
        val m = new Accumulator(width, data, rst)
        m.count
    }
}

// YOUR CODE HERE
???

In [ ]:
class AccumulatorInstMod(width: Int) extends Module {
    val io = IO(new Bundle {
        val data  = Input(UInt(width.W))
        val rst   = Input(Bool())
        val count = Output(UInt(width.W))
    })
    io.count := Accumulator(width, io.data, io.rst)
}

def testAccumulator: Boolean = {
    test(new AccumulatorInstMod(4)) { dut =>
        dut.io.data.poke(5.U)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.clock.step()
        
        dut.io.data.poke(6.U)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(5.U)
        dut.clock.step()
        
        dut.io.data.poke(7.U)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(11.U)
        dut.clock.step()
        
        dut.io.data.poke(0.U)
        dut.io.rst.poke(true.B)
        dut.io.count.expect(2.U)
        dut.clock.step()
        
        dut.io.count.expect(0.U)
    }
    true
}
assert(testAccumulator)

## Problem 4 (5 pts) - Valid
> Using your `Accumulator` class as a starting point, write a `ValidAccumulator` class that uses a `Valid[UInt]` as the input. The `ValidAccumulator` should only accumulate the incoming `data` if it is `valid`.

In [ ]:
object ValidAccumulator {
    def apply(width: Int, data: Valid[UInt], rst: Bool) = {
        val m = new ValidAccumulator(width, data, rst)
        m.count
    }
}

// YOUR CODE HERE
???

In [ ]:
class ValidAccumulatorInstMod(width: Int) extends Module {
    val io = IO(new Bundle {
        val data  = Input(Valid(UInt(width.W)))
        val rst   = Input(Bool())
        val count = Output(UInt(width.W))
    })
    io.count := ValidAccumulator(width, io.data, io.rst)
}

def testValidAccumulator: Boolean = {
    test(new ValidAccumulatorInstMod(4)) { dut =>
        dut.io.data.bits.poke(5.U)
        dut.io.data.valid.poke(false.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.clock.step()
        
        dut.io.data.bits.poke(5.U)
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.clock.step()
        
        dut.io.data.bits.poke(6.U)
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(5.U)
        dut.clock.step()
        
        dut.io.data.bits.poke(7.U)
        dut.io.data.valid.poke(false.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(11.U)
        dut.clock.step()
        
        dut.io.data.bits.poke(7.U)
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(11.U)
        dut.clock.step()
        
        dut.io.data.bits.poke(0.U)
        dut.io.data.valid.poke(false.B)
        dut.io.rst.poke(true.B)
        dut.io.count.expect(2.U)
        dut.clock.step()
        
        dut.io.count.expect(0.U)
    }
    true
}
assert(testValidAccumulator)

## Problem 5 (7 pts) - Decoupled
> Using your `ValidAccumulator` class as a starting point, write a `DecoupledAccumulator` class that uses a `Decoupled[UInt]` as the input. It must wait `coolDown` cycles after accepting an input before it can accept another. For example, if `coolDown = 1`, it can accept new numbers no faster than every other cycle. Like the previous Accumulator problems, add the proper reset logic. You can assume `coolDown > 0`. At start up or coming out of reset/rst, you need to wait the `coolDown` amount.

In [ ]:
object DecoupledAccumulator {
    def apply(width: Int, data: DecoupledIO[UInt], rst: Bool, coolDown: Int) = {
        require(coolDown > 0)
        val m = new DecoupledAccumulator(width, data, rst, coolDown)
        m.count
    }
}

// YOUR CODE HERE
???

In [ ]:
class DecoupledAccumulatorInstMod(width: Int, coolDown: Int) extends Module {
    val io = IO(new Bundle {
        val data  = Flipped(Decoupled(UInt(width.W)))
        val rst   = Input(Bool())
        val count = Output(UInt(width.W))
    })
    io.count := DecoupledAccumulator(width, io.data, io.rst, coolDown)
}

def testDecoupledAccumulator: Boolean = {
    test(new DecoupledAccumulatorInstMod(4, 1)) { dut =>
        dut.io.data.bits.poke(1.U) // ignored
        dut.io.data.valid.poke(true.B) 
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()

        dut.io.data.bits.poke(2.U) // accumed
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.io.data.ready.expect(true.B)
        dut.clock.step()

        dut.io.data.bits.poke(3.U) // ignored
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(2.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()

        dut.io.data.bits.poke(4.U) // accumed
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(2.U)
        dut.io.data.ready.expect(true.B)
        dut.clock.step()

        dut.io.data.bits.poke(5.U) // ignored since invalid
        dut.io.data.valid.poke(false.B) // doesn't count
        dut.io.rst.poke(false.B)
        dut.io.count.expect(6.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()

        dut.io.data.bits.poke(6.U) // ignored since invalid
        dut.io.data.valid.poke(false.B) // doesn't count
        dut.io.rst.poke(false.B)
        dut.io.count.expect(6.U)
        dut.io.data.ready.expect(true.B)
        dut.clock.step()

        dut.io.data.bits.poke(7.U) // accumed
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(6.U)
        dut.io.data.ready.expect(true.B)
        dut.clock.step()

        dut.io.count.expect(13.U)
        dut.io.rst.poke(true.B) // reset
        dut.clock.step()
        dut.io.count.expect(0.U)
    }
    test(new DecoupledAccumulatorInstMod(4, 2)) { dut =>
        dut.io.data.bits.poke(1.U) // ignored
        dut.io.data.valid.poke(true.B) 
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()

        dut.io.data.bits.poke(2.U) // ignored
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()

        dut.io.data.bits.poke(3.U) // accum
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(0.U)
        dut.io.data.ready.expect(true.B)
        dut.clock.step()

        dut.io.data.bits.poke(4.U) // ignored
        dut.io.data.valid.poke(true.B)
        dut.io.rst.poke(false.B)
        dut.io.count.expect(3.U)
        dut.io.data.ready.expect(false.B)
        dut.clock.step()
    }
    true
}
assert(testDecoupledAccumulator)